In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""Import modules."""
import contextlib

from src.robin.decision_model.terms import MembershipFS
from src.robin.decision_model.fuzzy_model import AcumulativeTSKFuzzyModel
from src.robin.decision_model.utils import read_json_variables, read_rules
from src.robin.decision_model.examples.passenger import read_supply, show_services


In [3]:
# Business passenger input files

passenger = {"variables": "../data/accumulative_tsk/business_passenger_files/passenger_vars.json",
             "rules": "../data/accumulative_tsk/business_passenger_files/business_passenger_rules.txt",
             "output": "../reports/accumulative_tsk_output/business_passenger.txt",
             "supply": "../data/accumulative_tsk/business_passenger_files/supply.csv"}


In [4]:
var_names, services = read_supply(passenger["supply"])
print(var_names)

['seat', 'service_departure_time', 'service_arrival_time', 'price']


In [5]:
show_services(var_names, services)

input_sequence = [services[service_id] for service_id in services]
labels = [e for e in services]

In [6]:
print(input_sequence)
print(labels)

[[1.0, 50.0, 40.0, 50.0], [1.0, 20.0, 20.0, 30.0], [2.0, 30.0, 20.0, 25.0], [3.0, 80.0, 75.0, 50.0], [2.0, 8.0, 3.0, 80.0]]
['service1', 'service2', 'service3', 'service4', 'service5']


In [10]:
# Define linguistic variables
variables = read_json_variables(passenger["variables"], var_names)
for name in variables:
    print(name, variables[name]['name'], end=', ')

print("\nVariables: ")
print(variables)

seat seat, service_departure_time service_departure_time, service_arrival_time service_arrival_time, price price, 
Variables: 
{'seat': {'name': 'seat', 'type': 'fuzzy', 'position': 0, 'support': [0, 100], 'sets': {'premium': <src.robin.decision_model.terms.MembershipFS object at 0x123117c50>, 'standard': <src.robin.decision_model.terms.MembershipFS object at 0x1231178d0>, 'tourist': <src.robin.decision_model.terms.MembershipFS object at 0x123116e50>}}, 'service_departure_time': {'name': 'service_departure_time', 'type': 'fuzzy', 'position': 1, 'support': [0, 100], 'sets': {'in_time': <src.robin.decision_model.terms.MembershipFS object at 0x123116c50>, 'close': <src.robin.decision_model.terms.MembershipFS object at 0x123115690>, 'far': <src.robin.decision_model.terms.MembershipFS object at 0x123114a90>}}, 'service_arrival_time': {'name': 'service_arrival_time', 'type': 'fuzzy', 'position': 2, 'support': [0, 100], 'sets': {'in_time': <src.robin.decision_model.terms.MembershipFS object a

In [8]:
# Read rules
rules = read_rules(passenger["rules"], variables)

In [9]:
# Define model
model = AcumulativeTSKFuzzyModel(rules)

# Show model
def inferencia(mod, input, labels):
    i = 0
    outputs = []
    for elem in mod.loop_tsk_inference(input):
        print('Type:', labels[i],
        '\nInput:', elem['Input'],
        '\nActives rules:', elem['actives_rules'], 
        '\nConsequents:', elem['consequents_values_list'],
        '\nMembership grades:', [ round(e,2) for e in elem['rule_membership_value_list'] ],
        '\nCons*Memb:', [ round(c*m,2) for c,m in zip(elem['consequents_values_list'],elem['rule_membership_value_list'])],
        '\nOutput:', round(elem['Output'],2), '\n')
        outputs.append( round(elem['Output'],2) )
        i += 1
    return outputs

# Ouput
tam = 10
with open(passenger["output"], 'w', encoding="utf-8") as f:
    with contextlib.redirect_stdout(f): # Salida a archivo de texto
        print(model)
        outputs = inferencia(model,input_sequence,labels)
        for size in range(0,tam*10,tam):
            l_aux = outputs[size:size+tam]
            if len(l_aux)==0:
                print(l_aux, 0.0)
            else:
                print(l_aux, round( sum(l_aux)/len(l_aux), 2) )
        aux_outputs = outputs[:]
        for pos in range(len(aux_outputs)):
            index = aux_outputs.index(max(aux_outputs))
            print( str(pos)+' Index:', index, '. Value:', aux_outputs[index] )
            aux_outputs[index] = -1.0
        

print(model)
outputs = inferencia(model,input_sequence,labels)
print( 'Best index:', outputs.index(max(outputs)) )

R0: IF (seat es premium) THEN <function read_rules.<locals>.<lambda> at 0x10ef8d4e0>
R1: IF (service_departure_time es in_time) & (service_arrival_time es in_time) THEN <function read_rules.<locals>.<lambda> at 0x10ef8d1c0>
R2: IF (price es standard) THEN <function read_rules.<locals>.<lambda> at 0x10ef8d580>

Type: service1 
Input: [1.0, 50.0, 40.0, 50.0] 
Actives rules: ['R0', 'R2'] 
Consequents: [35.0, 15.0] 
Membership grades: [1, 0.5] 
Cons*Memb: [35.0, 7.5] 
Output: 42.5 

Type: service2 
Input: [1.0, 20.0, 20.0, 30.0] 
Actives rules: ['R0', 'R1', 'R2'] 
Consequents: [35.0, 25.0, 15.0] 
Membership grades: [1, 0.25, 1] 
Cons*Memb: [35.0, 6.25, 15.0] 
Output: 56.25 

Type: service3 
Input: [2.0, 30.0, 20.0, 25.0] 
Actives rules: ['R0', 'R2'] 
Consequents: [35.0, 15.0] 
Membership grades: [1, 0.75] 
Cons*Memb: [35.0, 11.25] 
Output: 46.25 

Type: service4 
Input: [3.0, 80.0, 75.0, 50.0] 
Actives rules: ['R0', 'R2'] 
Consequents: [35.0, 15.0] 
Membership grades: [1, 0.5] 
Cons*Memb: 

In [13]:
# Define model
model = AcumulativeTSKFuzzyModel(rules)

# Show model
def inferencia(mod, input, labels):
    i = 0
    outputs = []
    for elem in [mod.tsk_inference(seq) for seq in input]:
        print(elem)
        """
        print('Type:', labels[i],
        '\nInput:', elem['Input'],
        '\nActives rules:', elem['actives_rules'], 
        '\nConsequents:', elem['consequents_values_list'],
        '\nMembership grades:', [ round(e,2) for e in elem['rule_membership_value_list'] ],
        '\nCons*Memb:', [ round(c*m,2) for c,m in zip(elem['consequents_values_list'],elem['rule_membership_value_list'])],
        '\nOutput:', round(elem['Output'],2), '\n')
        outputs.append( round(elem['Output'],2) )
        i += 1
        """
    return outputs

outputs = inferencia(model,input_sequence,labels)
outputs

{'Input': [1.0, 50.0, 40.0, 50.0], 'actives_rules': ['R0', 'R2'], 'consequents_values_list': [35.0, 15.0], 'rule_membership_value_list': [1, 0.5], 'Output': 42.5}
{'Input': [1.0, 20.0, 20.0, 30.0], 'actives_rules': ['R0', 'R1', 'R2'], 'consequents_values_list': [35.0, 25.0, 15.0], 'rule_membership_value_list': [1, 0.25, 1], 'Output': 56.25}
{'Input': [2.0, 30.0, 20.0, 25.0], 'actives_rules': ['R0', 'R2'], 'consequents_values_list': [35.0, 15.0], 'rule_membership_value_list': [1, 0.75], 'Output': 46.25}
{'Input': [3.0, 80.0, 75.0, 50.0], 'actives_rules': ['R0', 'R2'], 'consequents_values_list': [35.0, 15.0], 'rule_membership_value_list': [1, 0.5], 'Output': 42.5}
{'Input': [2.0, 8.0, 3.0, 80.0], 'actives_rules': ['R0', 'R1'], 'consequents_values_list': [35.0, 25.0], 'rule_membership_value_list': [1, 1], 'Output': 60.0}


[]

In [ ]:

# Ouput
tam = 10
with open(passenger["output"], 'w', encoding="utf-8") as f:
    with contextlib.redirect_stdout(f): # Salida a archivo de texto
        print(model)
        outputs = inferencia(model,input_sequence,labels)
        for size in range(0,tam*10,tam):
            l_aux = outputs[size:size+tam]
            if len(l_aux)==0:
                print(l_aux, 0.0)
            else:
                print(l_aux, round( sum(l_aux)/len(l_aux), 2) )
        aux_outputs = outputs[:]
        for pos in range(len(aux_outputs)):
            index = aux_outputs.index(max(aux_outputs))
            print( str(pos)+' Index:', index, '. Value:', aux_outputs[index] )
            aux_outputs[index] = -1.0
        

print(model)
outputs = inferencia(model,input_sequence,labels)
print( 'Best index:', outputs.index(max(outputs)) )